In [1]:
using EvolvingGraphs
using EvolvingGraphs.Centrality

In [56]:
workspace()

# Katz Motivation exmple 

We 

In [58]:
g = EvolvingGraph{Node{String}, Int}()

Directed EvolvingGraph 0 nodes, 0 static edges, 0 timestamps

In [59]:
add_bunch_of_edges!(g, [("A", "B", 1), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 3)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [60]:
edges(g)

5-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Float64},1}:
 Node(A)-1.0->Node(B) at time 1
 Node(A)-1.0->Node(C) at time 2
 Node(A)-1.0->Node(B) at time 2
 Node(C)-1.0->Node(A) at time 2
 Node(B)-1.0->Node(C) at time 3

In [61]:
katz(g)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.742301)
 (Node(B), 0.42943) 
 (Node(C), 0.514373)

In [62]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [63]:
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

In [27]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A","B", 1),("B","C", 2)])
katz(g1)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.621164)
 (Node(B), 0.621164)
 (Node(C), 0.477818)

In [31]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A","B", 2),("B","C", 1)])
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.621164)
 (Node(B), 0.621164)
 (Node(C), 0.477818)

In [32]:
nodes(g1)

3-element Array{EvolvingGraphs.Node{String},1}:
 Node(A)
 Node(B)
 Node(C)

In [33]:
A1 = adjacency_matrix(g1, 1)

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  1.0
 0.0  0.0  0.0

In [34]:
A2 = adjacency_matrix(g1, 2)

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [44]:
eig(A1)

([0.0, 0.0, 0.0], [1.0 0.0 0.0; 0.0 1.0 -1.0; 0.0 0.0 3.00625e-292])

In [35]:
alpha = 0.01
C = inv(I - alpha*A1)*inv(I - alpha*A2)
C*ones(3)

3-element Array{Float64,1}:
 1.01
 1.01
 1.0 

In [36]:
alpha = 0.01
C2 = inv(I - alpha*A2)*inv(I - alpha*A1)
C2*ones(3)

3-element Array{Float64,1}:
 1.0101
 1.01  
 1.0   

In [37]:
X = (I - alpha*A2)*(I - alpha*A1)
X\ones(3)

3-element Array{Float64,1}:
 1.01
 1.01
 1.0 

In [38]:
X2 = (I - alpha*A1)*(I - alpha*A2)
X2\ones(3)

3-element Array{Float64,1}:
 1.0101
 1.01  
 1.0   

In [41]:
v = ones(3)
x = (I -alpha*A2)\v
(I -alpha*A1)\x

3-element Array{Float64,1}:
 1.01
 1.01
 1.0 

In [49]:
B = [0 1 1; 0 0 0; 1 0 0]
eigvals(B)

3-element Array{Float64,1}:
  1.0
 -1.0
  0.0

# Katz

In [23]:
# using the Figure 1 from Paper: Communicability across evolving networks
simple_g_i = ["A", "B", "B", "C", "E", "A", "B", "F", "C", "F", "E", "A"]
simple_g_j = ["B", "F", "G", "E", "G", "B", "F", "D", "F", "G", "G", "B"]
simple_g_t = [1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3]
simple_g = evolving_graph_from_arrays(simple_g_i, simple_g_j, simple_g_t, is_directed = false)

Undirected EvolvingGraph 7 nodes, 24 static edges, 3 timestamps

In [14]:
g_i = [1, 1, 2]
g_j = [2, 3, 3]
t_1 = [1, 2, 3]
g = evolving_graph_from_arrays(g_i, g_j ,t_1)

Directed EvolvingGraph 3 nodes, 3 static edges, 3 timestamps

In [21]:
t_2 = [3, 2, 1]
g2 = evolving_graph_from_arrays(g_i, g_j, t_2)

Directed EvolvingGraph 3 nodes, 3 static edges, 3 timestamps

In [4]:
katz_rates = katz(g, 0.2)

3-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(1), 0.717069)
 (Node(2), 0.567174)
 (Node(3), 0.405124)

In [22]:
katz_rates = katz(g2, 0.2)

3-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(1), 0.667424)
 (Node(2), 0.572078)
 (Node(3), 0.476731)

In [ ]:
# being a fussy person, 1 decided to remind 2 again at timestamp t_2.
g_i = [1, 1, 2]
g_j = [2, 3, 3]
t_1 = [1, 2, 3]
g = evolving_graph_from_arrays(g_i, g_j ,t_1)

In [9]:
sorted_katz_rates = sort(katz_rates, by = x -> x[2], rev = true)

3-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(1), 0.717069)
 (Node(2), 0.567174)
 (Node(3), 0.405124)

In [6]:
"""
mode can be :static, :temporal,or :both
"""
function path_length(p::TemporalPath; mode::Symbol = :static)
    t_l = 0 #temporal length
    s_l = 0 #spaical length


    pairs = zip(p.nodes[1:end-1], p.nodes[2:end])
    for (n1, n2) in pairs
        k1 = node_key(n1)
        k2 = node_key(n2)
        t1 = node_timestamp(n1)
        t2 = node_timestamp(n2)

        s_l += k1 != k2 ? 1 : 0
        
        t_l += abs(t2 - t1)
    end
    switch_mode = Dict(:static => s_l,
                       :temporal => t_l,
                       :both => s_l + t_l)
    return Int(switch_mode[mode])
end

path_length

In [150]:
p = TemporalPath()
push!(p, TimeNode(1, "a", 2001))
push!(p, TimeNode(2, "b", 2001))
push!(p, TimeNode(3, "b", 2008))
push!(p, TimeNode(4, "a", 2008))
push!(p, TimeNode(5, "c", 2008))
push!(p, TimeNode(6, "b", 2009))

6-element Array{EvolvingGraphs.TimeNode,1}:
 TimeNode(a, 2001)
 TimeNode(b, 2001)
 TimeNode(b, 2008)
 TimeNode(a, 2008)
 TimeNode(c, 2008)
 TimeNode(b, 2009)

In [151]:
q = deepcopy(p)

TimeNode(a, 2001)->TimeNode(b, 2001)->TimeNode(b, 2008)->TimeNode(a, 2008)->TimeNode(c, 2008)->TimeNode(b, 2009)

In [5]:
function depth_first_impl(g::EvolvingGraph, i::TimeNode, j::TimeNode, p = TemporalPath(), shortest = TemporalPath(), all_paths = [])
    
    p = deepcopy(p)
    push!(p, i)
    #println("Current path: $(p)")
    
    
    if i == j
        return p, all_paths
    end
    
    for n in forward_neighbors(g, i)
        if !(n in p)
            if node_timestamp(n) <= node_timestamp(j)
                if shortest == TemporalPath() || (length(p) < length(shortest))
                    new_p, new_all_paths = depth_first_impl(g, n, j, p, shortest, all_paths)
                    if !(new_p in all_paths)
                        push!(all_paths, new_p)
                    end
                    if new_p != TemporalPath()
                        shortest = new_p
                    end
                end
            end
        end
    end
    return shortest, all_paths

end

depth_first_impl (generic function with 4 methods)

In [152]:
function single_source_shortest_path(g, i)
    
end

single_source_shortest_path (generic function with 1 method)

In [153]:
n1 = active_nodes(simple_g)[1]
n2 = active_nodes(simple_g)[10]
println(n1)
println(n2)

TimeNode(A, 1)
TimeNode(F, 2)


In [165]:
p, all_p = depth_first_impl(simple_g, n1, n2)

Current path: TimeNode(A, 1)
Current path: TimeNode(A, 1)->TimeNode(B, 1)
Current path: TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(F, 1)
Current path: TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(F, 1)->TimeNode(F, 2)
Current path: TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(G, 1)
Current path: TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(G, 1)->TimeNode(G, 2)
Current path: TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(B, 2)
Current path: TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(B, 2)->TimeNode(F, 2)
Current path: TimeNode(A, 1)->TimeNode(A, 2)
Current path: TimeNode(A, 1)->TimeNode(A, 2)->TimeNode(B, 2)
Current path: TimeNode(A, 1)->TimeNode(A, 2)->TimeNode(B, 2)->TimeNode(F, 2)


(TimeNode(A, 1)->TimeNode(A, 2)->TimeNode(B, 2)->TimeNode(F, 2), Any[TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(F, 1)->TimeNode(F, 2), TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(B, 2)->TimeNode(F, 2), TimeNode(A, 1)->TimeNode(A, 2)->TimeNode(B, 2)->TimeNode(F, 2)])

In [166]:
p

TimeNode(A, 1)->TimeNode(A, 2)->TimeNode(B, 2)->TimeNode(F, 2)

In [167]:
all_p

3-element Array{Any,1}:
 TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(F, 1)->TimeNode(F, 2)
 TimeNode(A, 1)->TimeNode(B, 1)->TimeNode(B, 2)->TimeNode(F, 2)
 TimeNode(A, 1)->TimeNode(A, 2)->TimeNode(B, 2)->TimeNode(F, 2)

In [25]:
# Let α be a number between 0 and 1. 
# We consider 3 cases:
# 1: only count static edges (1 static edge has length 1)
# 2: only conut causal edges (1 causal edge has length 1)
# 3. both

function simulated_katz(g::AbstractEvolvingGraph, alpha::Real)
    n = num_nodes(g)
    ans = active_nodes(g)
    ns = nodes(g)
    A = spzeros(Float64, n, n)
    
    for i in ans
        for j in ans
            v = 0.
            s, ps = depth_first_impl(g, i,j)
            for p in ps
                if length(p) > 0
                    v += alpha^(- path_length(p, mode = :static))
                end
            end
            i_index = g.node_indexof[node_key(i)]
            j_index = g.node_indexof[node_key(j)]
            A[i_index,j_index] += v
        end
    end
    rates = A*ones(n)
    r = [(node, rates[node.index]) for node in ns]
    # 
end

simulated_katz (generic function with 1 method)

In [26]:
r = simulated_katz(g, 0.2)

3-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(1), 16.0)
 (Node(2), 11.0)
 (Node(3), 1.0) 

In [16]:
active_nodes(g)

6-element Array{EvolvingGraphs.TimeNode{Int64,Int64},1}:
 TimeNode(1, 1)
 TimeNode(2, 1)
 TimeNode(1, 2)
 TimeNode(3, 2)
 TimeNode(2, 3)
 TimeNode(3, 3)

In [19]:
sorted = sort(r, by = x -> x[2], rev = true)

3-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(1), 2.24)
 (Node(2), 1.4) 
 (Node(3), 1.0) 

In [ ]:
function shortest_temporal_path(i::TimeNode, j::TimeNode)
    
end

In [ ]:
function simulated_betweeness(g::AbstractEvolvingGraph)
    
end

# PageRank

We use 

# Measure

Can compare changes in edges, and changes in rating. Match expectency.